# Data Pre-Processing Notebook

Commercial airlines play a crucial role in world transport and bring significant benefits to society by offering cheaper and easier long distance travel. After more than half a century of mainstream adoption (especially in the US), airline operations have seen major optimizations, and today function with excellent reliability even in the face of various engineering challenges. 

<br>Nonetheless, passengers occasionally experience inconveniences which occur in the form of flight delays or cancellations, disrupting an otherwise functioning system and causing significant inefficiencies. However, flight delays and cancellations are sometimes inevitable and play an important role in both profits and losses of airlines. A precise way of how to estimate flight delays and cancellations is thus critical for airlines, as the results of such a prediction could be applied to increase customer satisfaction and incomes of airlines. 

The data used in this project was collected from the Bureau of Transportation Statistics, Govt. of the USA. The [datasets](https://www.kaggle.com/divyansh22/flight-delay-prediction) used contain all the flights in the United States from January 2019 and January 2020. 

<br>The aim of this project is to investigate the main reasons and factors that contribute to flight delays and cancellations within the US during these months. We will establish research questions which will help us analyse the data provided from different angles, to gain a comprehensive overview of the problem. We will then focus on predictive analytics and try to predict what types of flights/airlines, which destinations and which days are most likely to be affected by delays and cancellations in the future.

After adding weather data to the initial raw [dataset](https://www.kaggle.com/divyansh22/flight-delay-prediction), this notebook provides the steps of data cleaning and pre-processing.

## Installations & Imports

In [1]:
import pandas as pd

## Loading data

We first load the data into variables `df19_raw`and `df20_raw` which will be used throughtout the project.

In [2]:
df19_raw = pd.read_csv("Jan_2019_ontime_weather_imputed.csv")
df20_raw = pd.read_csv("Jan_2020_ontime_weather_imputed.csv")

In [3]:
df19_raw.shape, df20_raw.shape

((583985, 35), (606497, 35))

The amount of flights during both months were quite similar, with January 2020 having only slightly more overall flights.


To make parts of the analysis easier to work with, we merge the two dataframes and add a YEAR column.

In [4]:
df19_raw['YEAR'] = 2019
df20_raw['YEAR'] = 2020

df = pd.concat([df19_raw, df20_raw]).reset_index(drop=True)
df.shape

(1190482, 36)

In [5]:
df[df.YEAR == 2019].shape

(583985, 36)

In [6]:
df.sample(1)

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,ORIGIN_wspd,ORIGIN_pres,DEST_tavg,DEST_tmin,DEST_tmax,DEST_prcp,DEST_snow,DEST_wspd,DEST_pres,YEAR
1101054,27,1,B6,20409,B6,N292JB,1646,11057,1105703,CLT,...,1.6,1019.2,1.7,-2.7,8.3,0.0,0.0,21.2,1018.6,2020


Now we have a dataset including all the original columns, weather data, as well as a `YEAR` column. The next step is to perform some data pre-processing.


## Data pre-processing

Within data pre-processing, we first have to look at what data types each column has, as different datasets might use different data formats and we need to make sure that the data in each column has the right data type so that it can be used in our further analysis. In terms of data cleaning, we will mainly look at the missing data (NaN values).

### Data types

Let's look at data types for each of the columns to get a better idea of the dataset.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1190482 entries, 0 to 1190481
Data columns (total 36 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   DAY_OF_MONTH           1190482 non-null  int64  
 1   DAY_OF_WEEK            1190482 non-null  int64  
 2   OP_UNIQUE_CARRIER      1190482 non-null  object 
 3   OP_CARRIER_AIRLINE_ID  1190482 non-null  int64  
 4   OP_CARRIER             1190482 non-null  object 
 5   TAIL_NUM               1187241 non-null  object 
 6   OP_CARRIER_FL_NUM      1190482 non-null  int64  
 7   ORIGIN_AIRPORT_ID      1190482 non-null  int64  
 8   ORIGIN_AIRPORT_SEQ_ID  1190482 non-null  int64  
 9   ORIGIN                 1190482 non-null  object 
 10  DEST_AIRPORT_ID        1190482 non-null  int64  
 11  DEST_AIRPORT_SEQ_ID    1190482 non-null  int64  
 12  DEST                   1190482 non-null  object 
 13  DEP_TIME               1167492 non-null  float64
 14  DEP_DEL15         

In [8]:
df

,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN,...,ORIGIN_wspd,ORIGIN_pres,DEST_tavg,DEST_tmin,DEST_tmax,DEST_prcp,DEST_snow,DEST_wspd,DEST_pres,YEAR
0,1,2,9E,20363,9E,N8688C,3280,11953,1195302,GNV,...,4.0,1021.9,15.9,12.9,17.9,NaN,0.000000,2.5,1016.6,2019
1,1,2,9E,20363,9E,N8928A,3301,11953,1195302,GNV,...,4.0,1021.9,15.9,12.9,17.9,NaN,0.000000,2.5,1016.6,2019
2,1,2,OO,20304,OO,N601XJ,3503,11953,1195302,GNV,...,4.0,1021.9,15.9,12.9,17.9,NaN,0.000000,2.5,1016.6,2019
3,1,2,9E,20363,9E,N181PQ,5016,11953,1195302,GNV,...,4.0,1021.9,15.9,12.9,17.9,NaN,0.000000,2.5,1016.6,2019
4,1,2,WN,19393,WN,N746SW,1272,13487,1348702,MSP,...,11.7,1030.6,15.9,12.9,17.9,NaN,0.000000,2.5,1016.6,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190477,31,5,G4,20368,G4,311NV,2188,14761,1476107,SFB,...,15.3,1025.8,-13.8,-18.1,-9.5,NaN,1428.405327,18.8,1021.6,2020
1190478,31,5,G4,20368,G4,222NV,813,14112,1411206,PIE,...,17.4,1025.0,-12.4,-17.1,-4.9,0.0,0.000000,5.0,1011.9,2020
1190479,31,5,G4,20368,G4,241NV,484,14761,1476107,SFB,...,15.3,1025.8,-12.4,-17.1,-4.9,0.0,0.000000,5.0,1011.9,2020
1190480,31,5,G4,20368,G4,312NV,2393,14761,1476107,SFB,...,15.3,1025.8,-8.7,-13.9,-2.8,NaN,879.005178,10.3,1029.3,2020


Instead of an object data type, we make the variables `OP_UNIQUE_CARRIER`, `OP_CARRIER`, `ORIGIN`, `DEST`, `DEP_TIME_BLK` categorical, as they represent a finite list of text values. 

In [9]:
categoricals = ['OP_UNIQUE_CARRIER', 'OP_CARRIER', 'ORIGIN', 'DEST', 'DEP_TIME_BLK']

In [10]:
df[categoricals] = df[categoricals].astype('category')

In [11]:
df['ORIGIN_factor'],origin_names = pd.factorize(df['ORIGIN'])
df['DEST_factor'],destination_names = pd.factorize(df['DEST'])
df['OP_CARRIER_factor'],op_carrier_names = pd.factorize(df['OP_CARRIER'])
df['OP_UNIQUE_CARRIER_factor'],op_unique_carrier_names = pd.factorize(df['OP_UNIQUE_CARRIER'])

Instead of a float data type, we make the variables `CANCELLED`, `DEP_DEL15` and `ARR_DEL15` integer.

In [39]:
df['CANCELLED'] = pd.to_numeric(df["CANCELLED"], downcast="integer")
df['DEP_DEL15'] = pd.to_numeric(df["DEP_DEL15"], downcast="integer")
df['ARR_DEL15'] = pd.to_numeric(df["ARR_DEL15"], downcast="integer")

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1190482 entries, 0 to 1190481
Data columns (total 40 columns):
 #   Column                    Non-Null Count    Dtype   
---  ------                    --------------    -----   
 0   DAY_OF_MONTH              1190482 non-null  int64   
 1   DAY_OF_WEEK               1190482 non-null  int64   
 2   OP_UNIQUE_CARRIER         1190482 non-null  category
 3   OP_CARRIER_AIRLINE_ID     1190482 non-null  int64   
 4   OP_CARRIER                1190482 non-null  category
 5   TAIL_NUM                  1187241 non-null  object  
 6   OP_CARRIER_FL_NUM         1190482 non-null  int64   
 7   ORIGIN_AIRPORT_ID         1190482 non-null  int64   
 8   ORIGIN_AIRPORT_SEQ_ID     1190482 non-null  int64   
 9   ORIGIN                    1190482 non-null  category
 10  DEST_AIRPORT_ID           1190482 non-null  int64   
 11  DEST_AIRPORT_SEQ_ID       1190482 non-null  int64   
 12  DEST                      1190482 non-null  category
 13  DEP_TIME    

### Data cleaning

In this part, we first look at how many missing values each column contains.

In [41]:
df.isna().sum()

DAY_OF_MONTH                    0
DAY_OF_WEEK                     0
OP_UNIQUE_CARRIER               0
OP_CARRIER_AIRLINE_ID           0
OP_CARRIER                      0
TAIL_NUM                     3241
OP_CARRIER_FL_NUM               0
ORIGIN_AIRPORT_ID               0
ORIGIN_AIRPORT_SEQ_ID           0
ORIGIN                          0
DEST_AIRPORT_ID                 0
DEST_AIRPORT_SEQ_ID             0
DEST                            0
DEP_TIME                    22990
DEP_DEL15                   23026
DEP_TIME_BLK                    0
ARR_TIME                    24107
ARR_DEL15                   26065
CANCELLED                       0
DIVERTED                        0
DISTANCE                        0
ORIGIN_tavg                  6151
ORIGIN_tmin                  6137
ORIGIN_tmax                  6137
ORIGIN_prcp                 55352
ORIGIN_snow                  6137
ORIGIN_wspd                  8289
ORIGIN_pres                 50285
DEST_tavg                    6159
DEST_tmin     

There are missing values, mostly among the weather data. However, there are also some NaN values related to departure and arrival times and delays, since in the case when a flight is cancelled, it cannot have a departure/arrivel time, as the flight itself did not happen. We inspect these times in the instances when a flight was cancelled. 

In [42]:
df.loc[df.CANCELLED == 1, ['DEP_TIME', 'ARR_TIME', 'CANCELLED']]

,DEP_TIME,ARR_TIME,CANCELLED
949,NaN,NaN,1
1434,NaN,NaN,1
1441,NaN,NaN,1
1448,NaN,NaN,1
1487,NaN,NaN,1
...,...,...,...
1190340,911.0,NaN,1
1190377,NaN,NaN,1
1190402,NaN,NaN,1
1190452,NaN,NaN,1


In [43]:
len(df.loc[df.CANCELLED == 1, ['DEP_TIME', 'ARR_TIME', 'CANCELLED']].loc[df.ARR_TIME.isnull()])

23625

We see that there are no arrival times for cancelled flights, which makes sense, but that might not be the case for departure times, as there could be the case that it can take off. We will use this information later for the predictive models, as we know that neither of these two columns will contribute to the predictions. One could encode a column with 'DEP_MISSING' or 'ARR_MISSING'. However, such columns are not available before the flight is cancelled, and will render the model useless in a real world setting.

Next we will inspect the missing values on weather data and to which extent dropping these will have an effect on the cancellations and delays.

In [44]:
org_weather = ['ORIGIN_tavg', 'ORIGIN_tmin', 'ORIGIN_tmax', 'ORIGIN_prcp', 'ORIGIN_snow', 'ORIGIN_wspd', 'ORIGIN_pres']
dest_weather = ['DEST_tavg', 'DEST_tmin', 'DEST_tmax', 'DEST_prcp', 'DEST_snow', 'DEST_wspd', 'DEST_pres']

In [45]:
df_clean_weather = df.dropna(subset=org_weather + dest_weather)

In [46]:
df_clean_weather.isna().sum()

DAY_OF_MONTH                    0
DAY_OF_WEEK                     0
OP_UNIQUE_CARRIER               0
OP_CARRIER_AIRLINE_ID           0
OP_CARRIER                      0
TAIL_NUM                     2823
OP_CARRIER_FL_NUM               0
ORIGIN_AIRPORT_ID               0
ORIGIN_AIRPORT_SEQ_ID           0
ORIGIN                          0
DEST_AIRPORT_ID                 0
DEST_AIRPORT_SEQ_ID             0
DEST                            0
DEP_TIME                    19842
DEP_DEL15                   19873
DEP_TIME_BLK                    0
ARR_TIME                    20810
ARR_DEL15                   22469
CANCELLED                       0
DIVERTED                        0
DISTANCE                        0
ORIGIN_tavg                     0
ORIGIN_tmin                     0
ORIGIN_tmax                     0
ORIGIN_prcp                     0
ORIGIN_snow                     0
ORIGIN_wspd                     0
ORIGIN_pres                     0
DEST_tavg                       0
DEST_tmin     

In [47]:
for col in ['CANCELLED', 'ARR_DEL15', 'DEP_DEL15']:
    print('=' * 15, col, '=' * 15)    
    print('Original values')
    print(df[col].value_counts())
    
    print('\nWith dropped rows')
    print(df_clean_weather[col].value_counts())
    print('=' * 15, col, '=' * 15)    
    print('')

=============== CANCELLED ===============
Original values
0    1166857
1      23625
Name: CANCELLED, dtype: int64

With dropped rows
0    1028560
1      20397
Name: CANCELLED, dtype: int64
=============== CANCELLED ===============

=============== ARR_DEL15 ===============
Original values
0.0    977089
1.0    187328
Name: ARR_DEL15, dtype: int64

With dropped rows
0.0    864604
1.0    161884
Name: ARR_DEL15, dtype: int64
=============== ARR_DEL15 ===============

=============== DEP_DEL15 ===============
Original values
0.0    986778
1.0    180678
Name: DEP_DEL15, dtype: int64

With dropped rows
0.0    873036
1.0    156048
Name: DEP_DEL15, dtype: int64
=============== DEP_DEL15 ===============



Dropping the values for weather data does not seem to change the proportions between cancellations and no cancellations, or delays and no delays. Thus we assume that it is safe to drop the missing values for weather data.

In [48]:
df = df_clean_weather

In [49]:
df19 = df.query('YEAR == 2019')
df20 = df.query('YEAR == 2020')

In [51]:
df19.to_csv("Jan_2019_ontime_weather_imputed_clean.csv", index=False)
df20.to_csv("Jan_2020_ontime_weather_imputed_clean.csv", index=False)